<a href="https://colab.research.google.com/github/MariiaSam/DS_homeWork/blob/main/HW8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import os

from sklearn import svm
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report



Mounted at /content/drive


In [4]:
# перевірка чи зчитується папка data з диску

# path_folder = '/content/drive/MyDrive/Colab Notebooks/Colab/HW8/data'
# path_folder


In [6]:
def combine_csv_from_subfolders(root_folder):

  df_combined = pd.DataFrame()

  for root, dirs, files in os.walk(root_folder):
    for file in files:
      if file.endswith('.csv'):
        file_path = os.path.join(root, file)
        df_temp = pd.read_csv(file_path)
        df_temp['activity'] = os.path.basename(root)
        df_combined = pd.concat([df_combined, df_temp], ignore_index=True)

  return df_combined

path_folder = '/content/drive/MyDrive/Colab Notebooks/Colab/HW8/data'

df_combined = combine_csv_from_subfolders(path_folder)

In [7]:
df_combined

,accelerometer_X,accelerometer_Y,accelerometer_Z,activity
0,0.009577,0.162806,9.768343,idle
1,-0.119710,0.181959,9.773131,idle
2,-0.181959,0.181959,9.782708,idle
3,-0.148441,0.215478,9.768343,idle
4,-0.114922,0.263362,9.744401,idle
...,...,...,...,...
193885,-2.959233,-14.820108,3.864242,walking
193886,-6.186617,-17.846380,2.518700,walking
193887,0.991200,-12.550406,-1.144428,walking
193888,-0.737414,-2.801216,0.933739,walking


In [8]:
time_features = df_combined.groupby('activity').agg({'accelerometer_X': ['mean', 'std', 'median', 'min', 'max', 'sum'],'accelerometer_Y': ['mean', 'std', 'median', 'min', 'max', 'sum'], 'accelerometer_Z': ['mean', 'std', 'median', 'min', 'max','sum']})

time_features.head()

accelerometer_X                                            \
                    mean       std    median        min        max   
activity                                                             
idle            0.096767  0.691892  0.234632 -10.448297   8.135497   
running         5.045027  9.952806  3.605668 -39.188293  39.188293   
stairs          0.353135  3.845949  0.483629 -11.238382  14.164097   
walking        -2.661092  4.648331 -2.298433 -26.815060  20.575770   

                        accelerometer_Y                                  \
                    sum            mean        std    median        min   
activity                                                                  
idle        3016.239864        2.225971   2.821235  0.028730  -2.595315   
running   515500.883817        8.082715  12.440356  7.967903 -39.188293   
stairs      1748.016884       -9.574353   4.922294 -8.944737 -25.828648   
walking  -147930.117572       -9.704326   5.933384 -9.974244 -39.188293   

                                   accelerometer_Z                      \
                max            sum            mean       std    median   
activity                                                                 
idle       9.916783   69383.508951        8.973247  1.211058  9.749189   
running   39.188293  825891.850448        1.387504  7.004976  0.407014   
stairs     4.994305  -47393.045005       -1.841333  5.297376 -1.589750   
walking   14.441825 -539463.499968       -1.126583  6.925928 -0.933739   

                                               
                min        max            sum  
activity                                       
idle      -0.296881  21.988348  279696.094989  
running  -39.188293  39.188293  141775.202970  
stairs   -38.086960  17.492037   -9114.596029  
walking  -39.169140  38.426937  -62626.741871

In [17]:
X = df_combined[['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z']]
y = df_combined['activity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [18]:
svm_model = SVC()
svm_model.fit(X_train, y_train)

SVC()

In [19]:
rf_model = RandomForestClassifier(n_jobs=-1)
rf_model.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1)

In [20]:
# https://scikit-learn.org/1.5/modules/generated/sklearn.metrics.classification_report.html
y_pred_svm = svm_model.predict(X_test)

accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f"Точність моделі SVM: {accuracy_svm:.4f}")


y_pred_rf = rf_model.predict(X_test)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Точність моделі Random Forest: {accuracy_rf:.4f}")

print('\n')
print('\n')

print("classification_report моделі SVM")
print(classification_report(y_test, y_pred_svm))

print("classification_report моделі Random Forest")
print(classification_report(y_test, y_pred_rf))



Точність моделі SVM: 0.8932
Точність моделі Random Forest: 0.9997




classification_report моделі SVM
              precision    recall  f1-score   support

        idle       0.96      0.99      0.97      9311
     running       0.93      0.90      0.92     30513
      stairs       1.00      0.00      0.01      1535
     walking       0.80      0.91      0.85     16808

    accuracy                           0.89     58167
   macro avg       0.92      0.70      0.69     58167
weighted avg       0.90      0.89      0.88     58167

classification_report моделі Random Forest
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00      9311
     running       1.00      1.00      1.00     30513
      stairs       1.00      1.00      1.00      1535
     walking       1.00      1.00      1.00     16808

    accuracy                           1.00     58167
   macro avg       1.00      1.00      1.00     58167
weighted avg       1.00      1.00      

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred_rf)

plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Матриця плутанини')
plt.colorbar()
tick_marks = range(len(set(y_test)))
plt.xticks(tick_marks, set(y_test))
plt.yticks(tick_marks, set(y_test))
plt.xlabel('Передбачений клас')
plt.ylabel('Справжній клас')
plt.show()

Модель Random Forest має кращі показники ніж модель SVM